In [2]:
import requests
import pandas as pd
import numpy as np

from datetime import date, datetime, timedelta
from src.utils import get_config, Logger

import src.Stock as Stock

config = get_config()
log = Logger()
url = f"https://financialmodelingprep.com/api/v3"
apikey = f"&apikey={config['credentials']['financialmodelingprep']}"

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [7]:
str(date.today())

'2021-10-20'

In [ ]:
Stock.Stock.get_earnings_events("COST")

In [4]:
ee = Stock.Stock.get_earnings_events("COST")
a = Stock.Stock.get_prices(ee[0])
Stock.Stock.get_rsi(a)
Stock.Stock.get_stdev(a)
Stock.Stock.get_3wk_return(a)

In [7]:
a.columns

Index(['date', 'open', 'high', 'low', 'close', 'volume', 'symbol', 'led',
       'surprise', 'days_since_led', 'rsi', 'rsi_lag1', 'rsi_lag2',
       'num_stdev', 'num_stdev_lag1', 'num_stdev_lag2', '3wk_adj_return'],
      dtype='object')

In [4]:
import pickle
# with open(f"data/model_data.pickle", 'rb') as f:
#     model_data = pickle.load(f)

In [15]:
model_data_distinct = model_data.drop_duplicates()

In [ ]:
with open(f"data/model_data.pickle", 'wb') as f:
    pickle.dump(model_data_distinct, f, protocol=pickle.HIGHEST_PROTOCOL)

In [5]:
from_date = "2005-01-01"
to_date = "2022-09-30"
log.info(f"Fetching updated data for SPY from {from_date} to {to_date}.")
r = requests.get(f"{url}/historical-price-full/SPY?from={from_date}&to={to_date}{apikey}")
r.json()
df = pd.json_normalize(r.json())


history = df['historical'].values[0]
hist_df = pd.json_normalize(history)
hist_df = hist_df.drop(['adjClose', 'change', 'unadjustedVolume', 'changePercent', 'vwap', 'label', 'changeOverTime'], axis=1)
hist_df = hist_df.sort_values('date', ascending=True)
#hist_df.set_index('date', drop=False, inplace=True)

close_4wk = hist_df['close'].shift(-20)
hist_df['4wk_return'] = 100 * (close_4wk - hist_df['close']) / hist_df['close']

close_3wk = hist_df['close'].shift(-15)
hist_df['3wk_return'] = 100 * (close_3wk - hist_df['close']) / hist_df['close']

close_2wk = hist_df['close'].shift(-10)
hist_df['2wk_return'] = 100 * (close_2wk - hist_df['close']) / hist_df['close']

close_1wk = hist_df['close'].shift(-5)
hist_df['1wk_return'] = 100 * (close_1wk - hist_df['close']) / hist_df['close']

spy_dict = {}
for idx, row in hist_df.iterrows():
    spy_dict[row['date']] = {"1k_return": row['1wk_return'], 
                             "2k_return": row['2wk_return'], 
                             "3k_return": row['3wk_return'], 
                             "4k_return": row['4wk_return'] }
    
with open(f"data/spy_returns.pickle", 'wb') as f:
    pickle.dump(spy_dict, f, protocol=pickle.HIGHEST_PROTOCOL)

10/01/2021 05:15:35 INFO: Fetching updated data for SPY from 2005-01-01 to 2022-09-30.


In [ ]:
d = pd.to_datetime(a.iloc[0].date) - pd.Timestamp.today()

In [ ]:
d> timedelta(weeks=13)

In [ ]:
def _to_Date(dt: str) -> datetime.date:
    return datetime.strptime(dt, '%Y-%m-%d').date()

In [ ]:
import pickle

In [13]:
with open(f"data/spy_returns.pickle", 'rb') as f:
    earnings_info = pickle.load(f)

In [ ]:
earnings_info

In [ ]:
str((pd.to_datetime('2021-04-28') - timedelta(days=90)).date())

In [10]:
str( date.today() - timedelta(days=120) )

'2021-06-22'

In [ ]:
import requests
import pandas as pd
import pickle
from datetime import date, datetime, timedelta
from src.utils import get_config, Logger

In [ ]:
config = get_config()
log = Logger()
today = date.today()
symbol='EGOV'
from_date='2020-11-01'

In [ ]:
log.info(f"Fetching updated data for {symbol}.")
r = requests.get(f"https://financialmodelingprep.com/api/v3/historical-price-full/{symbol}?from={from_date}&to={today}&apikey={config['credentials']['financialmodelingprep']}")
r.json()
df = pd.json_normalize(r.json())
history = df['historical'].values[0]
hist_df = pd.json_normalize(history)
hist_df 

#= hist_df.drop(['adjClose', 'change', 'unadjustedVolume', 'changePercent', 'vwap', 'label', 'changeOverTime'], axis=1)
#hist_df = hist_df.sort_values('date', ascending=True)

In [ ]:
exp_span = 60
ema = hist_df['close'].ewm(span=exp_span, adjust=False).mean().round(2)
emstd = hist_df['close'].ewm(span=exp_span, adjust=False).std().round(4)
boll_low = (ema - 2*emstd).round(2)
boll_sig = hist_df['low'] < boll_low
boll_sig_lag_1 = boll_sig.shift(1)
boll_sig_lag_2 = boll_sig.shift(2)
hist_df['boll'] = ~boll_sig & boll_sig_lag_1 & boll_sig_lag_2

In [ ]:
change = hist_df['close'].diff()
up = change.clip(lower=0)
down = -1 * change.clip(upper=0)
ema_up = up.ewm(span=13, adjust=False).mean()
ema_down = down.ewm(span=13, adjust=False).mean()
rs = ema_up/ema_down
rsi = 100 - (100/(1+rs))
rsi_sig = rsi < 25
rsi_sig_lag_1 = rsi_sig.shift(1)
rsi_sig_lag_2 = rsi_sig.shift(2)
hist_df['rsi_sig'] = ~rsi_sig & rsi_sig_lag_1 & rsi_sig_lag_2

In [ ]:
ema_sig_span = 4
ema_sig = hist_df['close'] < hist_df['close'].ewm(span=ema_sig_span, adjust=False).mean().round(2)
ema_sig_lag_1 = ema_sig.shift(1)
ema_sig_lag_2 = ema_sig.shift(2)
hist_df['ema_sig'] = ~ema_sig & ema_sig_lag_1 & ema_sig_lag_2

In [ ]:
buy_signal = hist_df[hist_df['boll'] & hist_df['rsi_sig'] & hist_df['ema_sig']]

In [ ]:
buy_dates = buy_signal['date'].values

In [ ]:
with open(f"data/signals.pickle", 'rb') as f:
    signals = pickle.load(f)

In [ ]:
signal_list = []

In [ ]:
for symbol, earnings_dates in signals.items():
    for earnings_date, signal_dates in earnings_dates.items():
        for signal in signal_dates:
            if signal > earnings_date:
                signal_list.append((symbol, signal))

In [ ]:
def get_open_price(symbol, date):
    from_date = date #str((pd.to_datetime(date) - timedelta(days=1)).date())
    to_date = str((pd.to_datetime(date) + timedelta(days=3)).date())
    
    r = requests.get(f"https://financialmodelingprep.com/api/v3/historical-price-full/{symbol}?from={from_date}&to={to_date}&apikey={config['credentials']['financialmodelingprep']}")
    r.json()
    df = pd.json_normalize(r.json())
    history = df['historical'].values[0]
    hist_df = pd.json_normalize(history)
    hist_df = hist_df.sort_values('date', ascending=True)
    
    if date in hist_df['date'].values:
        return hist_df[hist_df['date'] == date]['open'].values[0].round(2)
    else:
        return hist_df['open'].values[0].round(2)

In [ ]:
get_open_price("WMT", "2021-08-16")

In [ ]:
with open(f"{config['config']['data']}/signal_analysis.pickle", 'rb') as f:
            signal_perf = pickle.load(f)

In [ ]:
perf = pd.DataFrame(data=signal_perf)

In [ ]:
perf['surprise'] = ((perf['eps'] - perf['estimated']) / perf['estimated']).round(3) * 100

In [ ]:
signal_perf.sort_values

In [ ]:
signal_perf['gain_21'].mean()

In [ ]:
perf['gain_28'].mean()

In [ ]:
def _filter(x):
    if '2020-03' in x:
        return False
    else:
        return True

In [ ]:
perf['marketCap'] = [get_market_cap(x,y) for x,y in zip(perf['symbol'], perf['signal_date'])]

In [ ]:
filtered_perf = perf[[_filter(x) for x in perf['signal_date']]].sort_values('gain_21', ascending=False)

In [ ]:
beats = filtered_perf[filtered_perf['surprise'] < -70]

In [6]:
beats['gain_21'].mean()

NameError: name 'beats' is not defined

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(beats['surprise'], beats['gain_21'])
plt.xlim([-100,0])
plt.xlabel('surprise')
plt.ylim([-25,25])
plt.ylabel('% Gain Relative to S&P')

plt.show()

In [ ]:
def get_market_cap(symbol, date):
    r = requests.get(f"https://financialmodelingprep.com/api/v3/historical-market-capitalization/{symbol}?limit=1000&apikey={config['credentials']['financialmodelingprep']}")
    r.json()
    df = pd.json_normalize(r.json())
    df = df[df['date'] == date]
    try:
        cap = int(df['marketCap'].values[0]) / 1000000000  
    except Exception as e:
        cap = None
        
    return cap

In [19]:
url = "https://financialmodelingprep.com/api/v3/profile/"
symbol='KR'
apikey='?apikey=e0ed6052477ed6f2212a4fe50b5ac6c0'
r = requests.get(f"{url}{symbol}?limit=1000&apikey={config['credentials']['financialmodelingprep']}")
a=r.json()[0]
#df = pd.json_normalize(r.json())
#df


In [20]:
a

{'symbol': 'KR',
 'price': 39.07,
 'beta': 0.439682,
 'volAvg': 7559120,
 'mktCap': 29053937664,
 'lastDiv': 0.75,
 'range': '30.35-47.99',
 'changes': -0.139999,
 'companyName': 'The Kroger Co.',
 'currency': 'USD',
 'cik': '0000056873',
 'isin': 'US5010441013',
 'cusip': '501044101',
 'exchange': 'New York Stock Exchange',
 'exchangeShortName': 'NYSE',
 'industry': 'Grocery Stores',
 'website': 'http://www.thekrogerco.com',
 'description': "The Kroger Co. operates as a retailer in the United States. The company operates supermarkets, multi-department stores, marketplace stores, and price impact warehouse stores. Its combination food and drug stores offer natural food and organic sections, pharmacies, general merchandise, pet centers, fresh seafood, and organic produce; and multi-department stores provide apparel, home fashion and furnishings, outdoor living, electronics, automotive products, and toys. The company's marketplace stores offer full-service grocery, pharmacy, health and b

In [ ]:
signal_perf['marketCap'] = [get_market_cap(s,d) for s,d in (zip(signal_perf['symbol'].values, signal_perf['date'].values))]

In [ ]:
signal_perf.sort_values('marketCap', ascending=False)

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(signal_perf['marketCap'], signal_perf['gain_21'])
plt.xlim([0,100])
plt.xlabel('Market Cap (Billions)')
plt.ylim([-20,20])
plt.ylabel('% Gain Relative to S&P')

plt.show()